# TODO: 概要

# ライブラリのインポート

In [1]:
import os
import json
import asyncio
import datetime

from dotenv import load_dotenv, find_dotenv

from IPython.display import Image, display

from azure.identity.aio import DefaultAzureCredential
from azure.ai.agents.models import (
    FileInfo, FileSearchTool, VectorStore,
    CodeInterpreterTool, FilePurpose,
    ListSortOrder
)

from semantic_kernel.agents import (
    ChatCompletionAgent, ChatHistoryAgentThread,
    AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    ChatMessageContent, FunctionCallContent, FunctionResultContent, AuthorRole, TextContent
)


In [2]:
from semantic_kernel.agents import Agent, ChatCompletionAgent, HandoffOrchestration, OrchestrationHandoffs
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.agents.orchestration.group_chat import GroupChatOrchestration, RoundRobinGroupChatManager
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import ChatMessageContent

# 環境変数の取得

In [3]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")
FOUNDRY_FILE_SEARCH_AGENT_ID=os.getenv("FOUNDRY_FILE_SEARCH_AGENT_ID")

PG_HOST=os.getenv("PG_HOST")
PG_PORT=os.getenv("PG_PORT", "5432")
PG_DB=os.getenv("PG_DB")
PG_USER=os.getenv("PG_USER")
PG_PASS=os.getenv("PG_PASS")

# プラグインの作成

## PostgreSQL 接続情報

In [4]:
PG_CONFIG = {
    "user": os.getenv("PG_USER"),
    "password": os.getenv("PG_PASS"),
    "dbname": os.getenv("PG_DB"),  # psycopg2は'dbname'です（'database'不可）
    "host": os.getenv("PG_HOST"),
    "port": int(os.getenv("PGPORT", 5432)),
}

## 1. テーブル一覧・スキーマ取得プラグイン

In [5]:
import json
from typing import Annotated
import psycopg2
from semantic_kernel.functions import kernel_function



class PostgresSchemaPlugin:
    """
    テーブル一覧・スキーマ取得専用
    """
    def __init__(self):
        self.pg_config = PG_CONFIG

    def _get_connection(self):
        return psycopg2.connect(**self.pg_config)

    @kernel_function(
        name="get_tables",
        description="PostgreSQLデータベース内のテーブル一覧をJSON文字列で取得します。"
    )
    def get_tables(
        self,
    ) -> Annotated[str, "テーブル一覧を含むJSON文字列（例: {'tables': ['table1', 'table2']}）"]:
        with self._get_connection() as conn, conn.cursor() as cur:
            cur.execute("SELECT tablename FROM pg_tables WHERE schemaname = 'public';")
            tables = [row[0] for row in cur.fetchall()]
        return json.dumps({"tables": tables})

    @kernel_function(
        name="get_table_schema",
        description="指定したテーブルのスキーマ情報（カラム名、型など）をJSON文字列で取得します。"
    )
    def get_table_schema(
        self,
        table_name: Annotated[str, "スキーマ情報を取得したいテーブル名"]
    ) -> Annotated[str, "カラム情報を含むJSON文字列（例: {'columns': [{'name': 'id', 'type': 'integer'}, ...]}）"]:
        with self._get_connection() as conn, conn.cursor() as cur:
            cur.execute("""
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = %s;
            """, (table_name,))
            columns = [{"name": row[0], "type": row[1]} for row in cur.fetchall()]
        return json.dumps({"columns": columns})

## 2. SQL実行プラグイン

In [6]:
class PostgresQueryPlugin:
    """
    SQL実行専用
    """
    def __init__(self):
        self.pg_config = PG_CONFIG

    def _get_connection(self):
        return psycopg2.connect(**self.pg_config)

    @kernel_function(
        name="execute_sql",
        description="任意のSQL文を実行し、結果をJSON文字列で返します。（SELECTのみ対応を推奨）"
    )
    def execute_sql(
        self,
        sql: Annotated[str, "実行したいSQLクエリ（例: 'SELECT * FROM users'）"]
    ) -> Annotated[str, "クエリ結果のJSON文字列（例: {'rows': [...]}）"]:
        try:
            print(f"Executing SQL: {sql}")
            with self._get_connection() as conn, conn.cursor() as cur:
                cur.execute(sql)
                columns = [desc[0] for desc in cur.description] if cur.description else []
                rows = [dict(zip(columns, row)) for row in cur.fetchall()] if columns else []
            print(f"SQL executed successfully. Rows returned: {len(rows)}")
            # rowsを直接返すのではなく、明示的にrowsキーを含む辞書で返す
            return json.dumps({"rows": rows, "row_count": len(rows)})
        except Exception as e:
            error_msg = str(e)
            print(f"SQL execution failed: {error_msg}")
            raise Exception(error_msg)

# クライアントの初期化

In [7]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# エージェントの作成

## 構造化出力の事前準備

In [8]:
from pydantic import BaseModel
from typing import Any, List, Optional

class SqlExecutionResult(BaseModel):
    success: bool
    result: Optional[List[dict[str, Any]]]
    error_message: Optional[str]

In [9]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.functions.kernel_arguments import KernelArguments


# 構造化出力の設定
settings = AzureChatPromptExecutionSettings()

# 辞書形式でresponse_formatを設定（簡素化版）
response_format_dict = {
    "type": "json_schema",
    "json_schema": {
        "name": "SqlExecutionResult",
        "schema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["success", "error"]
                },
                "result": {
                    "type": "string",
                    "description": "JSON string containing the SQL execution result or error message"
                }
            },
            "required": ["status", "result"],
            "additionalProperties": False
        },
        "strict": True
    }
}

print("Response format dict:", response_format_dict)

settings.response_format = response_format_dict

Response format dict: {'type': 'json_schema', 'json_schema': {'name': 'SqlExecutionResult', 'schema': {'type': 'object', 'properties': {'status': {'type': 'string', 'enum': ['success', 'error']}, 'result': {'type': 'string', 'description': 'JSON string containing the SQL execution result or error message'}}, 'required': ['status', 'result'], 'additionalProperties': False}, 'strict': True}}


In [10]:
# --- SQL作成エージェント ---
sql_generation_agent = ChatCompletionAgent(
    name="SQLGenerationAgent",
    description="ユーザーの自然言語質問からPostgreSQL用のSQLクエリを生成し、必要に応じて修正も行うエージェントです。",
    instructions=(
        "あなたはユーザーの質問に基づいて、PostgreSQLデータベースで実行可能なSQLクエリ（主にSELECT文）を生成する役割です。\n"
        "【必ず守ること】\n"
        "1. SQLクエリを生成する前に、まずデータベース内のテーブル一覧を取得してください。\n"
        "2. 次に、関連しそうなテーブルごとにカラム情報も取得し、スキーマ構成を十分に把握してください。\n"
        "3. 取得したテーブル・カラム情報（スキーマ情報）を必ず参照したうえで、PostgreSQL用の正しいSQLを生成してください。\n"
        "4. SQLを生成した後、SQL実行エージェントによって実行されます。\n"
        "5. もしSQL実行時にエラー（テーブルやカラムが存在しない、構文エラーなど）が発生した場合は、エラーメッセージとスキーマ情報を再確認し、原因を特定してSQLを修正してください。\n"
        "6. 不明点やスキーマに疑問がある場合は、ツールを活用して追加で情報取得を行い、十分に情報を得てから再度SQLを生成してください。\n"
        "7. 最終的に正しいSQLが完成したら、そのSQL文のみを返してください。\n"
        "【注意事項】\n"
        "・SQLインジェクションや危険なクエリ生成は厳禁です。\n"
        "・不要なコメントや説明文は出力せず、SQL文のみを返してください。\n"
        "・必ず実際のテーブル名とカラム名を確認してからSQLを生成してください。"
    ),
    service=azure_completion_service,
    plugins=[PostgresQueryPlugin()]
)

# --- SQL実行エージェント ---
sql_execution_agent = ChatCompletionAgent(
    service=AzureChatCompletion(
        service_id="azure_sql_execution_agent",
        deployment_name=AZURE_DEPLOYMENT_NAME,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY
    ),
    name="SQLExecutionAgent",
    description="PostgreSQLのSQL文を実行し、構造化JSONで返すエージェント。",
    instructions=(
        "あなたは与えられたSQLクエリを、必ずそのままPostgreSQLで実行する役割です。\n"
        "【実行手順】\n"
        "1. 提供されたSQLクエリを、execute_sql関数を使用して実行してください。\n"
        "2. execute_sql関数の結果は JSON文字列形式で返されます。\n"
        "3. 実行結果に基づいて、以下の構造化JSON形式で必ず回答してください：\n\n"
        "成功時:\n"
        "{\n"
        '  "status": "success",\n'
        '  "result": "execute_sql関数から返されたJSON文字列をそのまま格納"\n'
        "}\n\n"
        "エラー時:\n"
        "{\n"
        '  "status": "error",\n'
        '  "result": "具体的なエラーメッセージ"\n'
        "}\n\n"
        "【重要】\n"
        "・必ずexecute_sql関数を呼び出してSQLを実行してください。\n"
        "・execute_sql関数の結果（JSON文字列）をresultフィールドにそのまま文字列として格納してください。\n"
        "・危険なSQL（データ破壊やセキュリティリスクを伴うもの）は実行せず、status を error にして理由をresultに記載してください。\n"
        "・構造化JSONのみを返し、余計な説明やコメントは不要です。"
    ),
    plugins=[PostgresQueryPlugin()],
    arguments=KernelArguments(settings=settings)
)

# オーケストレーションの作成

## カスタムマネージャーの作成

In [11]:
from semantic_kernel.agents import GroupChatManager, BooleanResult, StringResult, MessageResult
from semantic_kernel.contents import ChatMessageContent, ChatHistory
import json

class SqlGroupChatManager(GroupChatManager):
    def __init__(self, max_rounds: int = 10):
        super().__init__(max_rounds=max_rounds)
        self.__dict__['current_index'] = 0
    
    async def _generate_response(self, user_question: str, sql_results: list) -> str:
        """SQLの実行結果を基に回答を生成"""
        try:
            # 自然言語回答生成用のAzure OpenAIサービスを作成
            completion_service = AzureChatCompletion(
                service_id="generate_response_service",
                deployment_name=AZURE_DEPLOYMENT_NAME,
                endpoint=AZURE_OPENAI_ENDPOINT,
                api_key=AZURE_OPENAI_API_KEY
            )
            
            # プロンプトを構築
            prompt = f"""
            以下のユーザーの質問に対して、SQLクエリの実行結果を基に自然言語で回答を生成してください。

            【ユーザーの質問】
            {user_question}

            【SQLクエリの実行結果】
            {sql_results}

            【回答形式】
            - 簡潔で分かりやすい日本語で回答してください
            - データの件数や主要な傾向を含めてください
            - 具体的な数値やデータがある場合は適切に言及してください
            - 「【質問】」や「【回答】」などの見出しは含めず、回答内容のみを返してください

            回答:
            """

            chat_history = ChatHistory()
            chat_history.add_user_message(prompt)
            
            response = await completion_service.get_chat_message_contents(
                chat_history=chat_history,
                settings=AzureChatPromptExecutionSettings(
                    max_tokens=500,
                    temperature=0.3
                )
            )
            
            generated_response = response[0].content if response else "回答の生成に失敗しました。"
            
            return generated_response
            
        except Exception as e:
            print(f"自然言語回答生成エラー: {e}")
            return f"SQL実行結果から自然言語での回答生成に失敗しました。"

    from typing_extensions import override
    @override
    async def filter_results(self, chat_history: ChatHistory) -> MessageResult:
        """SQL結果があれば自然言語で要約、なければエラーを返す"""
        # ユーザーのタスクを取得（最初のユーザーメッセージから）
        user_task = ""
        for msg in chat_history.messages:
            if hasattr(msg, 'role') and msg.role == AuthorRole.USER:
                user_task = msg.content
                break

        # SQL実行成功レスポンスとSQL文を逆順で探索
        sql_result = None
        executed_sql = ""
        sql_execution_found = False
        
        for msg in reversed(chat_history.messages):
            if getattr(msg, "name", None) == "SQLExecutionAgent" and not sql_execution_found:
                try:
                    content_json = json.loads(msg.content)
                    
                    if content_json.get("status") == "success":
                        # 新しいフォーマット: resultフィールドはJSON文字列
                        result_string = content_json.get("result", "")
                                                
                        try:
                            # JSON文字列をパース
                            result_data = json.loads(result_string)
                            
                            # result_dataは {"rows": [...], "row_count": N} の形式を期待
                            if isinstance(result_data, dict) and "rows" in result_data:
                                sql_result = result_data["rows"]
                            elif isinstance(result_data, list):
                                sql_result = result_data
                            else:
                                sql_result = result_data
                            sql_execution_found = True
                        except Exception as parse_error:
                            print(f"内部JSON解析エラー: {parse_error}")
                            print(f"パース対象文字列の最初の100文字: {result_string[:100]}...")
                            continue
                    elif content_json.get("status") == "error":
                        print(f"デバッグ: SQLエラーを検出: {content_json.get('result', 'Unknown error')}")
                        # エラーの場合は処理を続行してSQL修正を試みる
                        continue
                except Exception as e:
                    print(f"JSON解析エラー: {e}")
                    print(f"エラー内容の最初の100文字: {msg.content[:100]}...")
                    continue
        
        # 最後に実行されたSQLを検索（SQLExecutionAgentの関数呼び出しから取得）
        for msg in reversed(chat_history.messages):
            if getattr(msg, "name", None) == "SQLExecutionAgent":
                # メッセージのitemsからFunctionCallContentを探す
                for item in getattr(msg, 'items', []):
                    if hasattr(item, 'name') and item.name == 'execute_sql':
                        if hasattr(item, 'arguments') and item.arguments:
                            try:
                                args_dict = json.loads(item.arguments) if isinstance(item.arguments, str) else item.arguments
                                if 'sql' in args_dict:
                                    executed_sql = args_dict['sql']
                                    break
                            except Exception as e:
                                print(f"SQL引数の解析エラー: {e}")
                                # argumentsがdictの場合も試す
                                if isinstance(item.arguments, dict) and 'sql' in item.arguments:
                                    executed_sql = item.arguments['sql']
                                    break
                if executed_sql:
                    break
        
        # SQLが見つからない場合、SQLGenerationAgentからも確認
        if not executed_sql:
            for msg in reversed(chat_history.messages):
                if getattr(msg, "name", None) == "SQLGenerationAgent":
                    # SQLGenerationAgentの最後のメッセージがSQLクエリの可能性
                    content = getattr(msg, 'content', '')
                    if content and ('SELECT' in content.upper() or 'INSERT' in content.upper() or 'UPDATE' in content.upper() or 'DELETE' in content.upper()):
                        executed_sql = content.strip()
                        break
        # 結果が見つからなければそのまま
        if sql_result is None:
            summary = f"SQLの実行結果が見つかりませんでした。"
            # エラーの場合もSQLを追加（実行されたSQLがある場合）
            if executed_sql.strip():
                summary += f"\n\n# Executed SQL\n{executed_sql.strip()}"
        else:
            # SQLの実行結果を基に自然言語での回答を生成
            if isinstance(sql_result, list) and len(sql_result) > 0:
                # データが存在する場合の自然言語回答生成
                natural_response = await self._generate_response(user_task, sql_result)
                # 回答に実行したSQLも追加
                summary = f"{natural_response}\n\n# Executed SQL\n{executed_sql.strip()}"
            else:
                summary = f"該当するデータは見つかりませんでした。"
                # データなしの場合もSQLを追加
                if executed_sql.strip():
                    summary += f"\n\n# Executed SQL\n{executed_sql.strip()}"

        # assistantロールのChatMessageContentで返す
        return MessageResult(
            result=ChatMessageContent(
                role="assistant",
                content=summary
            ),
            reason="タスク内容とSQL結果をもとに回答を生成"
        )

    @override
    async def select_next_agent(self, chat_history: ChatHistory, participant_descriptions: dict[str, str]) -> StringResult:
        """次に実行するエージェントを選択（ラウンドロビン方式）"""
        next_agent = list(participant_descriptions.keys())[self.__dict__['current_index']]
        self.__dict__['current_index'] = (self.__dict__['current_index'] + 1) % len(participant_descriptions)
        return StringResult(result=next_agent, reason="Round-robin selection.")
    
    @override
    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        """SQLExecutionAgentの出力に {'status':'success'} を返したときに終了する"""
        for msg in reversed(chat_history.messages):
            if getattr(msg, "name", None) == "SQLExecutionAgent":
                try:
                    content_json = json.loads(msg.content)
                    if content_json.get("status") == "success":
                        return BooleanResult(
                            result=True,
                            reason="SQLExecutionAgentがstatus:successを返したため終了"
                        )
                except Exception as e:
                    print(f"終了条件判定でJSON解析エラー: {e}")
                    continue
        
        return BooleanResult(
            result=False,
            reason="まだSQL実行が成功していないため継続"
        )

    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        # ユーザー入力は不要（自動進行）
        return BooleanResult(
            result=False,
            reason="自動進行のためユーザー入力は不要"
        )

## オーケストレーション実行

In [ ]:
# エージェントのレスポンスを処理するコールバック関数
def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"{message.name}: {message.content}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"Calling '{item.name}' with arguments '{item.arguments}'")
        if isinstance(item, FunctionResultContent):
            print(f"Result from '{item.name}' is '{item.result}'")


# グループチャットのオーケストレーションを定義
async def run_group_chat():
    # マネージャーを作成
    manager = SqlGroupChatManager(max_rounds=10)
    
    # オーケストレーションを構成
    group_chat_orchestration = GroupChatOrchestration(
        members=[sql_generation_agent, sql_execution_agent],
        manager=manager,
        # agent_response_callback=agent_response_callback,
    )

    # ランタイムを初期化＆開始
    runtime = InProcessRuntime()
    runtime.start()

    # タスクを実行
    orchestration_result = await group_chat_orchestration.invoke(
        # task="ユーザー一覧を取得してください",
        task="2024年のユーザー別の売り上げ実績を取得してください。",
        runtime=runtime,
    )

    # 結果を取得
    value = await orchestration_result.get()
    print(f"# Result\n{value}")

    # ランタイムを停止
    await runtime.stop_when_idle()



In [13]:
await run_group_chat()

Executing SQL: SELECT table_name FROM information_schema.tables WHERE table_schema='public'
SQL executed successfully. Rows returned: 6
Executing SQL: SELECT column_name FROM information_schema.columns WHERE table_name = 'users'
SQL executed successfully. Rows returned: 3
Executing SQL: SELECT column_name FROM information_schema.columns WHERE table_name = 'orders'
SQL executed successfully. Rows returned: 3
Executing SQL: SELECT column_name FROM information_schema.columns WHERE table_name = 'order_details'
SQL executed successfully. Rows returned: 4
Executing SQL: SELECT column_name FROM information_schema.columns WHERE table_name = 'products'
SQL executed successfully. Rows returned: 4
Executing SQL: SELECT u.user_id, u.user_name, SUM(od.quantity * od.price) AS total_sales
FROM users u
JOIN orders o ON u.user_id = o.user_id
JOIN order_details od ON o.order_id = od.order_id
WHERE o.order_date >= '2024-01-01' AND o.order_date < '2025-01-01'
GROUP BY u.user_id, u.user_name
ORDER BY total